# Statistical Arbitrage - Data Collection



## 1. Setup & Configuration

Initialize LSEG session and define indices and dates for data collection.


In [ ]:
import lseg.data as ld
import pandas as pd
import pickle

ld.open_session(app_key="ff25e00926b145099c655332d969c9762ed6ff7b")

<lseg.data.session.Definition object at 0x11e654a50 {name='workspace'}>

In [74]:
indices = ['.BFX','.OMXC20 ', '.OMXH25','.SBF120','.GDAXI','.AEX','.AMX','.OBX','.OMXS30']
date = ['2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31', '2023-12-31', '2024-12-31']


## 2. Load Index Constituent Data

Load constituent data (RIC, Name) for each index across all specified dates.


In [ ]:
def load_all_index_data(indices, dates):
    """Load index constituent data for all indices and dates."""
    results = {}
    failed_combinations = []
    
    for index in indices:
        results[index] = []
        print(f"\n{'='*50}\nLoading data for {index}\n{'='*50}")
        
        for date in dates:
            try:
                print(f"Trying to load {index} with date {date}...")
                df = ld.get_data(
                    universe=[index],
                    fields=[
                        f'TR.IndexConstituentRIC(SDate={date})',
                        f'TR.IndexConstituentName(SDate={date})'
                    ]
                )
                
                if df is not None and not df.empty:
                    df['Date'] = date
                    df['Index'] = index
                    results[index].append((df, date))
                    print(f"✓ Successfully loaded {index} with date {date} ({len(df)} constituents)")
                else:
                    print(f"✗ No data returned for {index} with date {date}")
                    failed_combinations.append((index, date))
                    
            except Exception as e:
                print(f"✗ Error loading {index} with date {date}: {str(e)}")
                failed_combinations.append((index, date))
                continue
        
        if not results[index]:
            print(f"⚠ No data loaded for {index} with any date")
        else:
            print(f"✓ Successfully loaded {len(results[index])} dates for {index}")
    
    print(f"\n{'='*60}\nSUMMARY\n{'='*60}")
    for index in indices:
        print(f"{index}: {len(results[index])}/{len(dates)} dates loaded")
    if failed_combinations:
        print(f"\n⚠ {len(failed_combinations)} failed combinations")
    print(f"{'='*60}\n")
    
    return results

all_data = load_all_index_data(indices, date)


Loading data for .BFX
Trying to load .BFX with date 2019-12-31...
✓ Successfully loaded .BFX with date 2019-12-31 (20 constituents)
Trying to load .BFX with date 2020-12-31...
✓ Successfully loaded .BFX with date 2020-12-31 (20 constituents)
Trying to load .BFX with date 2021-12-31...
✓ Successfully loaded .BFX with date 2021-12-31 (20 constituents)
Trying to load .BFX with date 2022-12-31...
✓ Successfully loaded .BFX with date 2022-12-31 (20 constituents)
Trying to load .BFX with date 2023-12-31...
✓ Successfully loaded .BFX with date 2023-12-31 (20 constituents)
Trying to load .BFX with date 2024-12-31...
✓ Successfully loaded .BFX with date 2024-12-31 (20 constituents)
✓ Successfully loaded 6 dates for .BFX

Loading data for .OMXC20 
Trying to load .OMXC20  with date 2019-12-31...
✓ Successfully loaded .OMXC20  with date 2019-12-31 (20 constituents)
Trying to load .OMXC20  with date 2020-12-31...
✓ Successfully loaded .OMXC20  with date 2020-12-31 (20 constituents)
Trying to load 

An error occurred while requesting URL('http://localhost:9030/api/udf').
	ReadTimeout('timed out')


✗ Error loading .OMXS30 with date 2021-12-31: timed out
Trying to load .OMXS30 with date 2022-12-31...
✓ Successfully loaded .OMXS30 with date 2022-12-31 (29 constituents)
Trying to load .OMXS30 with date 2023-12-31...
✓ Successfully loaded .OMXS30 with date 2023-12-31 (30 constituents)
Trying to load .OMXS30 with date 2024-12-31...
✓ Successfully loaded .OMXS30 with date 2024-12-31 (30 constituents)
✓ Successfully loaded 4 dates for .OMXS30

SUMMARY
.BFX: 6/6 dates loaded
.OMXC20 : 6/6 dates loaded
.OMXH25: 6/6 dates loaded
.SBF120: 6/6 dates loaded
.GDAXI: 6/6 dates loaded
.AEX: 6/6 dates loaded
.AMX: 6/6 dates loaded
.OBX: 6/6 dates loaded
.OMXS30: 4/6 dates loaded

⚠ 2 failed combinations (out of 54 total)



## 3. Combine & Process Data

Combine all loaded data into a single dataframe.


In [ ]:
all_dfs = []
for index, data_list in all_data.items():
    for df, date_used in data_list:
        all_dfs.append(df)

combined_df = pd.concat(all_dfs, ignore_index=True)

print(f"Combined dataframe shape: {combined_df.shape}")
print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
print(f"Unique indices: {combined_df['Index'].nunique()}")
print(f"Unique dates: {combined_df['Date'].nunique()}")
display(combined_df.head(10))

combined_df.to_csv('all_data.csv', index=False)

Combined dataframe shape: (1913, 5)
Date range: 2019-12-31 to 2024-12-31
Unique indices: 9
Unique dates: 6


,Instrument,Constituent RIC,Constituent Name,Date,Index
0,.BFX,SOLB.BR,SOLVAY ORD,2019-12-31,.BFX
1,.BFX,ACKB.BR,ACKERMANS V.HAAR ORD,2019-12-31,.BFX
2,.BFX,ARGX.BR,ARGEN X NV,2019-12-31,.BFX
3,.BFX,PROX.BR,PROXIMUS,2019-12-31,.BFX
4,.BFX,INGA.AS,ING GROEP ORD,2019-12-31,.BFX
5,.BFX,ONTEX.BR,ONTEX GRP NV,2019-12-31,.BFX
6,.BFX,APAM.AS,APERAM ORD,2019-12-31,.BFX
7,.BFX,TNET.BR^J23,TELENET GRP HLDG ORD,2019-12-31,.BFX
8,.BFX,WDPP.BR,WDP-SICAFI REIT,2019-12-31,.BFX
9,.BFX,GBLB.BR,GROUPE BRUXELLES LAMBERT ORD,2019-12-31,.BFX


,Instrument,Constituent RIC,Constituent Name,Date,Index
0,.BFX,SOLB.BR,SOLVAY ORD,2019-12-31,.BFX
1,.BFX,ACKB.BR,ACKERMANS V.HAAR ORD,2019-12-31,.BFX
2,.BFX,ARGX.BR,ARGEN X NV,2019-12-31,.BFX
3,.BFX,PROX.BR,PROXIMUS,2019-12-31,.BFX
4,.BFX,INGA.AS,ING GROEP ORD,2019-12-31,.BFX
...,...,...,...,...,...
1908,.OMXS30,SHBa.ST,SVENSKA HANDELSBANKEN CLS A ORD,2024-12-31,.OMXS30
1909,.OMXS30,VOLVb.ST,VOLVO CL B ORD,2024-12-31,.OMXS30
1910,.OMXS30,KINVb.ST,KINNEVIK STK B ORD,2024-12-31,.OMXS30
1911,.OMXS30,ABB.ST,ABB LTD ORD,2024-12-31,.OMXS30


### 3.1 Add OMXS30 Data (Manual Fix)

OMXS30 data requires separate handling for specific dates.


In [ ]:
dates = ['2020-12-31', '2021-12-31']
omsx30_data = {}
for date in dates:
    omsx30_data[date] = ld.get_data(
        universe=['.OMXS30'],
        fields=[
            f'TR.IndexConstituentRIC(SDate={date})',
            f'TR.IndexConstituentName(SDate={date})'
        ]
    )

omsx30 = pd.concat(list(omsx30_data.values()))
omsx30['Date'] = pd.to_datetime(dates[0])
omsx30['Index'] = '.OMXS30'
omsx30 = omsx30[['Instrument', 'Constituent RIC', 'Constituent Name', 'Date', 'Index']]

combined_df = pd.concat([combined_df, omsx30], ignore_index=True)
combined_df.to_csv('namelist.csv', index=False)

,Instrument,Constituent RIC,Constituent Name,Date,Index
0,.BFX,SOLB.BR,SOLVAY ORD,2019-12-31,.BFX
1,.BFX,ACKB.BR,ACKERMANS V.HAAR ORD,2019-12-31,.BFX
2,.BFX,ARGX.BR,ARGEN X NV,2019-12-31,.BFX
3,.BFX,PROX.BR,PROXIMUS,2019-12-31,.BFX
4,.BFX,INGA.AS,ING GROEP ORD,2019-12-31,.BFX
...,...,...,...,...,...
1968,.OMXS30,SHBa.ST,SVENSKA HANDELSBANKEN CLS A ORD,2020-12-31 00:00:00,.OMXS30
1969,.OMXS30,VOLVb.ST,VOLVO CL B ORD,2020-12-31 00:00:00,.OMXS30
1970,.OMXS30,KINVb.ST,KINNEVIK STK B ORD,2020-12-31 00:00:00,.OMXS30
1971,.OMXS30,ABB.ST,ABB ORD,2020-12-31 00:00:00,.OMXS30


## 4. Create Namelist

Generate namelist with RIC, Date, and Company Name.


In [ ]:
namelist = pd.read_csv('all_data.csv')
namelist = namelist[['Constituent RIC', 'Date', 'Constituent Name']]
namelist.rename(columns={'Constituent RIC': 'RIC', 'Constituent Name': 'Name'}, inplace=True)
namelist['Date'] = pd.to_datetime(namelist['Date'], format='mixed')
namelist.to_csv('namelist.csv', index=False)

print(f"Namelist shape: {namelist.shape}")
print(f"Unique RICs: {namelist['RIC'].nunique()}")
display(namelist.head())


,RIC,Date,Name
0,SOLB.BR,2019-12-31,SOLVAY ORD
1,ACKB.BR,2019-12-31,ACKERMANS V.HAAR ORD
2,ARGX.BR,2019-12-31,ARGEN X NV
3,PROX.BR,2019-12-31,PROXIMUS
4,INGA.AS,2019-12-31,ING GROEP ORD
...,...,...,...
1968,SHBa.ST,2020-12-31 00:00:00,SVENSKA HANDELSBANKEN CLS A ORD
1969,VOLVb.ST,2020-12-31 00:00:00,VOLVO CL B ORD
1970,KINVb.ST,2020-12-31 00:00:00,KINNEVIK STK B ORD
1971,ABB.ST,2020-12-31 00:00:00,ABB ORD


## 5. Download Additional Fields

Fetch ISIN codes and TRBC Industry Group classifications for all stocks.


In [ ]:
indexlist = namelist['RIC'].tolist()
print(f"Total RICs to process: {len(indexlist)}")

['SOLB.BR',
 'ACKB.BR',
 'ARGX.BR',
 'PROX.BR',
 'INGA.AS',
 'ONTEX.BR',
 'APAM.AS',
 'TNET.BR^J23',
 'WDPP.BR',
 'GBLB.BR',
 'AGES.BR',
 'COFB.BR',
 'GLPG.AS',
 'UMI.BR',
 'ABI.BR',
 'COLR.BR',
 'SOF.BR',
 'KBC.BR',
 'UCB.BR',
 'BAR.BR',
 'SOLB.BR',
 'ACKB.BR',
 'ARGX.BR',
 'PROX.BR',
 'INGA.AS',
 'APAM.AS',
 'TNET.BR^J23',
 'WDPP.BR',
 'AOO.BR',
 'GBLB.BR',
 'AGES.BR',
 'COFB.BR',
 'GLPG.AS',
 'UMI.BR',
 'ABI.BR',
 'COLR.BR',
 'SOF.BR',
 'KBC.BR',
 'UCB.BR',
 'BAR.BR',
 'SOLB.BR',
 'ACKB.BR',
 'ARGX.BR',
 'PROX.BR',
 'APAM.AS',
 'TNET.BR^J23',
 'ELI.BR',
 'WDPP.BR',
 'AOO.BR',
 'MLXS.BR',
 'GBLB.BR',
 'AGES.BR',
 'COFB.BR',
 'GLPG.AS',
 'UMI.BR',
 'ABI.BR',
 'COLR.BR',
 'SOF.BR',
 'KBC.BR',
 'UCB.BR',
 'SOLB.BR',
 'ACKB.BR',
 'ARGX.BR',
 'PROX.BR',
 'APAM.AS',
 'ELI.BR',
 'WDPP.BR',
 'AOO.BR',
 'GBLB.BR',
 'AGES.BR',
 'IETB.BR',
 'COFB.BR',
 'GLPG.AS',
 'UMI.BR',
 'VGP1.BR',
 'ABI.BR',
 'COLR.BR',
 'SOF.BR',
 'KBC.BR',
 'UCB.BR',
 'SOLB.BR',
 'ACKB.BR',
 'SYENS.BR',
 'ARGX.BR',
 'PRO

In [ ]:
def download_additional_fields(ric_list, fields=['TR.ISINCode', 'TR.TRBCIndustryGroup'], batch_size=100):
    """Download additional fields for a list of RICs in batches."""
    print(f"Downloading {len(fields)} fields for {len(ric_list)} stocks...")
    print(f"Fields: {', '.join(fields)}")
    
    all_results = []
    failed_rics = []
    
    for i in range(0, len(ric_list), batch_size):
        batch = ric_list[i:i+batch_size]
        batch_num = i // batch_size + 1
        total_batches = (len(ric_list) + batch_size - 1) // batch_size
        
        print(f"\nProcessing batch {batch_num}/{total_batches} ({len(batch)} RICs)...")
        
        try:
            df = ld.get_data(universe=batch, fields=fields)
            
            if df is not None and not df.empty:
                all_results.append(df)
                print(f"✓ Successfully downloaded data for batch {batch_num}")
            else:
                print(f"✗ No data returned for batch {batch_num}")
                failed_rics.extend(batch)
                
        except Exception as e:
            print(f"✗ Error downloading batch {batch_num}: {str(e)}")
            failed_rics.extend(batch)
            continue
    
    if all_results:
        additional_data = pd.concat(all_results, ignore_index=True)
        
        print(f"\n{'='*60}\nSUMMARY\n{'='*60}")
        print(f"Total stocks requested: {len(ric_list)}")
        print(f"Successfully downloaded: {len(ric_list) - len(failed_rics)}")
        if failed_rics:
            print(f"Failed: {len(failed_rics)}")
        print(f"Result shape: {additional_data.shape}\n{'='*60}\n")
        
        return additional_data
    else:
        print("\n⚠ No data was downloaded successfully")
        return None

ind_isin = download_additional_fields(indexlist)


Fields: TR.ISINCode, TR.TRBCIndustryGroup

Processing batch 1/20 (100 RICs)...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/lseg/data/_access_layer/_containers.py:62:UserWarning: You have duplicated instruments in your input. Output will contain unique instruments only.


✓ Successfully downloaded data for batch 1

Processing batch 2/20 (100 RICs)...
✓ Successfully downloaded data for batch 2

Processing batch 3/20 (100 RICs)...
✓ Successfully downloaded data for batch 3

Processing batch 4/20 (100 RICs)...
✓ Successfully downloaded data for batch 4

Processing batch 5/20 (100 RICs)...
✓ Successfully downloaded data for batch 5

Processing batch 6/20 (100 RICs)...
✓ Successfully downloaded data for batch 6

Processing batch 7/20 (100 RICs)...
✓ Successfully downloaded data for batch 7

Processing batch 8/20 (100 RICs)...
✓ Successfully downloaded data for batch 8

Processing batch 9/20 (100 RICs)...
✓ Successfully downloaded data for batch 9

Processing batch 10/20 (100 RICs)...
✓ Successfully downloaded data for batch 10

Processing batch 11/20 (100 RICs)...
✓ Successfully downloaded data for batch 11

Processing batch 12/20 (100 RICs)...
✓ Successfully downloaded data for batch 12

Processing batch 13/20 (100 RICs)...
✓ Successfully downloaded data fo

In [ ]:
ind_isin.rename(columns={'Instrument': 'RIC'}, inplace=True)
ind_isin.to_csv('isinlist.csv')

print(f"ISIN list shape: {ind_isin.shape}")
print(f"Unique RICs: {ind_isin['RIC'].nunique()}")
display(ind_isin.head())

## 6. Merge Data

Combine namelist with ISIN and industry information.


In [ ]:
merged_data = pd.merge(namelist, ind_isin, on='RIC', how='left')
merged_data = merged_data.drop_duplicates().reset_index(drop=True)

print(f"Merged dataset shape: {merged_data.shape}")
display(merged_data.head(10))

merged_data.to_csv('merged_data.csv', index=False)



Merged dataset shape: (1884, 5)


,RIC,Date,Name,ISIN Code,TRBC Industry Group Name
0,SOLB.BR,2019-12-31,SOLVAY ORD,BE0003470755,Chemicals
1,ACKB.BR,2019-12-31,ACKERMANS V.HAAR ORD,BE0003764785,Construction & Engineering
2,ARGX.BR,2019-12-31,ARGEN X NV,NL0010832176,Biotechnology & Medical Research
3,PROX.BR,2019-12-31,PROXIMUS,BE0003810273,Telecommunications Services
4,INGA.AS,2019-12-31,ING GROEP ORD,NL0011821202,Banking Services
...,...,...,...,...,...
1879,ABB.ST,2020-12-31,ABB ORD,CH0012221716,"Machinery, Tools, Heavy Vehicles, Trains & Ships"
1880,SWEDa.ST,2020-12-31,SWEDBANK ORD,SE0000242455,Banking Services
1881,SINCH.ST,2020-12-31,SINCH ORD,SE0016101844,Software & IT Services
1882,EVOG.ST,2020-12-31,EVOLUTION AB ORD,SE0012673267,Hotels & Entertainment Services


## 7. Export Final Data

Create pickle file containing namelist and isinlist for easy loading.


In [ ]:
namelist_final = pd.read_csv('namelist.csv')
isinlist_final = pd.read_csv('isinlist.csv', index_col=0)

alldata = {
    'namelist': namelist_final,
    'isinlist': isinlist_final
}

with open('alldata.pkl', 'wb') as f:
    pickle.dump(alldata, f)

print("✓ Created pickle file 'alldata.pkl'")
print(f"\nContents:")
print(f"  - namelist: {namelist_final.shape}")
print(f"  - isinlist: {isinlist_final.shape}")

with open('alldata.pkl', 'rb') as f:
    loaded = pickle.load(f)
    print(f"\n✓ Verified: {len(loaded)} dataframes in pickle file")


✓ Created pickle file 'alldata.pkl'

Contents:
  - namelist: (1973, 3) (RIC, Date, Name)
  - isinlist: (1241, 3) (RIC, ISIN Code, TRBC Industry Group Name)

✓ Verified: Successfully loaded pickle file with 2 dataframes


---

## Output Files Summary

This notebook generates the following files:

1. **`all_data.csv`** - Combined constituent data for all indices and dates
2. **`namelist.csv`** - List of all RICs with dates and company names
3. **`isinlist.csv`** - ISIN codes and TRBC Industry Groups for all stocks
4. **`merged_data.csv`** - Merged dataset combining namelist with ISIN/industry data
5. **`alldata.pkl`** - Pickle file containing namelist and isinlist dataframes
